In [ ]:
!pip install tensorflow==2.16.1

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential, Input
from keras.layers import Dense, LSTM, Normalization
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

matplotlib.rcParams['figure.figsize'] = [9, 6]
sns.set_theme(style="darkgrid")
seed = 0
tf.random.set_seed(seed)
np.random.seed(seed)

print(tf.__version__)

2.16.1


In [ ]:
path='https://docs.google.com/spreadsheets/d/1dmDOMzkkUKqNJidsPmD4tjvtiu4u9WAV0ngfZL6MfkE/export?format=csv'
df = pd.read_csv(path)

In [ ]:
df.tail(5)

,Time,h2_light_100,h2_light_200,h2_light_300
19,121,24.485,25.14,20.78
20,126,23.315,25.26,23.06
21,138,26.440,30.19,24.70
22,141,28.805,39.44,23.90
23,144,33.790,51.98,23.22


In [ ]:
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Time          24 non-null     int64  
 1   h2_light_100  24 non-null     float64
 2   h2_light_200  24 non-null     float64
 3   h2_light_300  24 non-null     float64
dtypes: float64(3), int64(1)
memory usage: 896.0 bytes


In [ ]:
df['light_100'] = pd.Series(['100' for x in range(len(df.index))])
df['light_200'] = pd.Series(['200' for x in range(len(df.index))])
df['light_300'] = pd.Series(['300' for x in range(len(df.index))])
df.tail(5)

,Time,h2_light_100,h2_light_200,h2_light_300,light_100,light_200,light_300
19,121,24.485,25.14,20.78,100,200,300
20,126,23.315,25.26,23.06,100,200,300
21,138,26.440,30.19,24.70,100,200,300
22,141,28.805,39.44,23.90,100,200,300
23,144,33.790,51.98,23.22,100,200,300


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Time          24 non-null     int64  
 1   h2_light_100  24 non-null     float64
 2   h2_light_200  24 non-null     float64
 3   h2_light_300  24 non-null     float64
 4   light_100     24 non-null     int64  
 5   light_200     24 non-null     int64  
 6   light_300     24 non-null     int64  
dtypes: float64(3), int64(4)
memory usage: 1.4 KB


In [ ]:
df1 = df[['h2_light_100', 'light_100']]
df2 = df[['h2_light_200', 'light_200']]
df3 = df[['h2_light_300', 'light_300']]
df3.tail(5)

,h2_light_300,light_300
19,20.78,300
20,23.06,300
21,24.70,300
22,23.90,300
23,23.22,300


In [ ]:
df2.columns = ['h2_light_100', 'light_100']
df2.tail(5)

,h2_light_100,light_100
19,25.14,200
20,25.26,200
21,30.19,200
22,39.44,200
23,51.98,200


In [ ]:
df4 = pd.concat([df1, df2], ignore_index=True)

In [ ]:
df3.columns = ['h2_light_100', 'light_100']
df3.tail(5)

,h2_light_100,light_100
19,20.78,300
20,23.06,300
21,24.70,300
22,23.90,300
23,23.22,300


In [ ]:
df5 = pd.concat([df4, df3], ignore_index=True)
print(df5)

    h2_light_100  light_100
0           0.00        100
1           0.00        100
2           0.00        100
3           0.00        100
4           0.00        100
..           ...        ...
67         20.78        300
68         23.06        300
69         24.70        300
70         23.90        300
71         23.22        300

[72 rows x 2 columns]


In [ ]:
df5.columns = ['H2', 'Light Intensity']
df5

,H2,Light Intensity
0,0.00,100
1,0.00,100
2,0.00,100
3,0.00,100
4,0.00,100
...,...,...
67,20.78,300
68,23.06,300
69,24.70,300
70,23.90,300


In [ ]:
df5.sort_values('H2', inplace=True)
df5.head(20)

,H2,Light Intensity
0,0.0,100
31,0.0,200
30,0.0,200
29,0.0,200
28,0.0,200
27,0.0,200
26,0.0,200
25,0.0,200
24,0.0,200
48,0.0,300


In [ ]:
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val loss')
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.legend()
  # plt.grid(True)
  plt.show()